In [20]:
!pip install google-cloud-aiplatform --upgrade --user --quiet

In [2]:
import pandas as pd
from IPython.display import Markdown, display
from vertexai.language_models import TextGenerationModel

2023-09-28 19:01:20.902395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 19:01:25.611939: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-28 19:01:25.613079: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [3]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [4]:
def query_model(prompt, model_predictor):    
    response = model_predictor.predict(
        prompt=prompt
    )
    
    return {'output': response.text} 

In [6]:
import pandas as pd

eval_set = pd.read_json("evaluation_v2.jsonl",lines=True)
eval_set.head()

,prompt,output_text
0,Classify the following domain name into one of...,constructionservices
1,Classify the following domain name into one of...,constructionservices
2,Classify the following domain name into one of...,personal
3,Classify the following domain name into one of...,personal
4,Classify the following domain name into one of...,artists


In [9]:
# get result from the base model

import time

# Create an empty list to store the results
results_list = []

for i, row in eval_set.iterrows():
    result = query_model(prompt = row["prompt"], model_predictor = generation_model)
    results_list.append(result)
    if i % 60 == 0:
        time.sleep(60)
    
# Create a new DataFrame from the results list
results_list = pd.DataFrame(results_list)

results_list

,output
0,constructionservices
1,homeservices
2,transportationservice
3,bbapparel
4,entertainment
...,...
1643,bbapparel
1644,bbtechnology
1645,food
1646,civilengineers


In [12]:
results_list.to_json('PaLM2-text-bison-IC.json')

In [21]:
# load fine tuned model
model_tuned = TextGenerationModel.from_pretrained("text-bison@001")
model_tuned = model_tuned.get_tuned_model("projects/793496231204/locations/us-central1/models/3422988604475965440")

In [22]:
# get result from the fine tuned model

import time

# Create an empty list to store the results
results_list_tuned = []

for i, row in eval_set.iterrows():
    result = query_model(prompt = row["prompt"], model_predictor = model_tuned)
    results_list_tuned.append(result)
    # if i % 60 == 0:
    #     time.sleep(60)
    
# Create a new DataFrame from the results list
results_list_tuned = pd.DataFrame(results_list_tuned)

results_list_tuned

,output
0,buildingremodelingrepair
1,homeservices
2,
3,personal_blog
4,entertainmentagency
...,...
1643,fashion
1644,bbbusinessservices
1645,food
1646,civilengineers


In [24]:
results_list_tuned.to_json('PaLM2-text-bison-tuned-IC.json')

In [23]:
# analyze result

results_list.rename(columns={'output': 'text_bison_output'}, inplace=True)
results_list_tuned.rename(columns={'output': 'text_bison_tuned_output'}, inplace=True)

df = eval_set

combined_df = pd.concat([df, 
                         results_list, 
                         results_list_tuned,], axis=1)
combined_df.head()

,prompt,output_text,text_bison_output,text_bison_tuned_output
0,Classify the following domain name into one of...,constructionservices,constructionservices,buildingremodelingrepair
1,Classify the following domain name into one of...,constructionservices,homeservices,homeservices
2,Classify the following domain name into one of...,personal,transportationservice,
3,Classify the following domain name into one of...,personal,bbapparel,personal_blog
4,Classify the following domain name into one of...,artists,entertainment,entertainmentagency


In [28]:
accuracy_bison = (combined_df['text_bison_output'] == combined_df['output_text']).mean()
print(f"Accuracy of the original model: {accuracy_bison * 100:.2f}%")

accuracy_bison_tuned = (combined_df['text_bison_tuned_output'] == combined_df['output_text']).mean()
print(f"Accuracy of the fine tuned model: {accuracy_bison_tuned * 100:.2f}%")

Accuracy of the original model: 5.22%
Accuracy of the fine tuned model: 15.29%
